In [1]:
import numpy as np
from matplotlib import pyplot
from geojson.feature import *
import os
from tqdm import tqdm
import h3
import geopandas as gpd
import json
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
import folium
from folium import Map, Marker, GeoJson
import branca.colormap as cm
from branca.colormap import LinearColormap
import vaex
import pandas as pd
from math import pi, cos, sin
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from shapely.geometry import Polygon
from folium.features import Choropleth
from branca.colormap import linear
from sklearn.preprocessing import MinMaxScaler

In [2]:
# demand
# time per mile
# idle time

In [3]:
df = vaex.open('./data/trips_prepared.hdf5')

In [4]:
resolution = 7

In [5]:
def convert_to_h3(lat,long):
    return h3.geo_to_h3(lat, long, resolution)

In [6]:
df['pickup_hex'] = df.apply(convert_to_h3, [df['pickup_centroid_latitude'], df['pickup_centroid_longitude']])

In [7]:
df.head()

#,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,trip_start_hour,trip_end_hour,trip_start_4h_period,trip_end_4h_period,pickup_hex
0,1b545d0be27b195076e5d2a7d4fc0ebef6ef20ac,'56a1119c6ca57e39525cf06829f9ecff553cf4b5ac24821...,2017-01-01 00:00:00.000000,2017-01-01 00:00:00.000000,240,1.3,1.70318e+10,1.70318e+10,32,28,6.25,0,0,0,6.25,Cash,Taxi Affiliation Services,41.881,-87.6327,41.8853,-87.6572,0,0,0,0,872664c1affffff
1,c3ee40051c5643c54064a8b9df3686d2d411518d,'7987cfe5e3b2c9869665f70fe35dbe7423e7fc5852f2d6f...,2017-01-01 00:00:00.000000,2017-01-01 00:15:00.000000,1500,0.4,1.70318e+10,1.70311e+10,32,8,22.75,4.85,0,1.5,29.1,Credit Card,Blue Ribbon Taxi Association Inc.,41.881,-87.6327,41.8983,-87.6208,0,0,0,0,872664c1affffff
2,29f30c8a6bbc028f5d63aa18c708d31b47267a9f,'0b269137bd94b55212311226d9e1554423e12bcc16a432c...,2017-01-01 00:00:00.000000,2017-01-01 00:30:00.000000,1620,0.9,1.70318e+10,1.70311e+10,32,8,12.25,2.85,0,2,17.1,Credit Card,Taxi Affiliation Services,41.881,-87.6327,41.8925,-87.6262,0,0,0,0,872664c1affffff
3,55ae4fa955e6bc8705b995248c34f4183d848c27,'d1d781b6807176ed53fbb9d0010dd0a87e29c073984bda6...,2017-01-01 00:00:00.000000,2017-01-01 00:00:00.000000,540,1,1.70311e+10,1.70311e+10,8,8,6.75,0,0,2,8.75,Cash,Dispatch Taxi Affiliation,41.9028,-87.6261,41.8909,-87.6189,0,0,0,0,872664c1effffff
4,ce908cd441e28d08dfb22bf168102beabb5e16e4,'f7630edc68a131c4a97382dd93206949da80ec1443d8da7...,2017-01-01 00:00:00.000000,2017-01-01 00:15:00.000000,480,0.5,1.70311e+10,1.70311e+10,8,8,6.25,2,0,1.5,9.75,Credit Card,Taxi Affiliation Services,41.8925,-87.6262,41.8909,-87.6189,0,0,0,0,872664c1effffff
5,fd25560e18dab95225999c264586a331dc3283f5,'9ff153898c75dbd960a207b660a71f92f79b666dcec98e1...,2017-01-01 00:00:00.000000,2017-01-01 00:30:00.000000,1980,3.4,1.70318e+10,1.70311e+10,32,8,16.75,3.65,0,1.5,21.9,Credit Card,Taxi Affiliation Services,41.881,-87.6327,41.895,-87.6197,0,0,0,0,872664c1affffff
6,2504e0bf058ff310ac4c197618a843f88fc1c0c6,'d3a5f1452b4a1db2a547e7a0aabfe28d67c62b0e40155fa...,2017-01-01 00:00:00.000000,2017-01-01 00:15:00.000000,960,5.9,1.70313e+10,1.70311e+10,32,6,17.5,0,0,1,18.5,Cash,Northwest Management LLC,41.8774,-87.622,41.9427,-87.6518,0,0,0,0,872664c1affffff
7,4ca1087133dfd8bde2483cb2aaa609fff2654f7c,'0083fcde0fb490b4ec424c63d1d750378f6ad11154d1d4a...,2017-01-01 00:00:00.000000,2017-01-01 00:15:00.000000,780,0.1,1.70312e+10,1.70311e+10,24,8,9.5,2.1,0,1,12.6,Credit Card,Taxi Affiliation Services,41.8997,-87.665,41.892,-87.6319,0,0,0,0,872664cadffffff
8,04bc3089a41c33afc9b6cbf5a9beac4fed91da7c,'785ef52cff80fe78a7ee899453036c0be6ac8fef4d10068...,2017-01-01 00:00:00.000000,2017-01-01 00:00:00.000000,120,0.4,1.70318e+10,1.70318e+10,32,32,4.25,2,0,1,7.25,Credit Card,Chicago Medallion Leasing INC,41.881,-87.6327,41.881,-87.6327,0,0,0,0,872664c1affffff
9,e09945909ae154d6b0ce433468baddcf79f79973,'b329626db30d23db0e6620de7d1a3036dd3c2ca3e139fd4...,2017-01-01 00:00:00.000000,2017-01-01 00:00:00.000000,180,0.7,1.70318e+10,1.70318e+10,32,28,5,0,0,2,7,Cash,Dispatch Taxi Affiliation,41.881,-87.6327,41.8853,-87.6572,0,0,0,0,872664c1affffff


In [8]:
pickup_count = df.groupby(by='pickup_hex').agg({'Count': vaex.agg.count('pickup_hex')}) 
pickup_count.sort(by='Count', ascending= False)

#,pickup_hex,Count
0,872664c1effffff,5116354
1,872664c1affffff,3137329
2,87275934effffff,690410
3,872664cadffffff,548494
4,872664c1bffffff,244801
...,...,...
63,872664ccaffffff,1
64,872664cd3ffffff,1
65,872664c85ffffff,1
66,872664cceffffff,1


In [9]:
def h3_to_poly(row):
    points = h3.h3_to_geo_boundary(
        row, geo_json=True)
    return Polygon(points)

In [10]:
pickup_count["Geometry"] = pickup_count["pickup_hex"].apply(
    h3_to_poly)

In [11]:
pickup_count.head()

#,pickup_hex,Count,Geometry
0,872664d88ffffff,3629,"'<POLYGON ((-87.676 41.997, -87.689 41.988, -87...."
1,872664ca8ffffff,1923,"'<POLYGON ((-87.69 41.899, -87.702 41.891, -87.6..."
2,872664d83ffffff,11,"'<POLYGON ((-87.718 42.029, -87.731 42.02, -87.7..."
3,872664cc8ffffff,2,"'<POLYGON ((-87.59 41.741, -87.603 41.733, -87.5..."
4,872664d8cffffff,2269,"'<POLYGON ((-87.667 42.017, -87.68 42.009, -87.6..."
5,872664ca4ffffff,253,"'<POLYGON ((-87.714 41.972, -87.727 41.963, -87...."
6,872664cf0ffffff,14,"'<POLYGON ((-87.656 41.846, -87.669 41.838, -87...."
7,872664cabffffff,317,"'<POLYGON ((-87.698 41.878, -87.711 41.87, -87.7..."
8,872664cc4ffffff,7,"'<POLYGON ((-87.615 41.814, -87.627 41.806, -87...."
9,872664cacffffff,46957,"'<POLYGON ((-87.681 41.919, -87.694 41.911, -87...."


In [13]:
df_pickup = pickup_count.to_pandas_df()

In [14]:
#Convert Count to float so it can be later serialized in json
df_pickup['Count'] = df_pickup['Count'].astype("float")

In [15]:
#Create a custom legend, which will display the connection between color and pickup-count on the map later
legend_pick = linear.YlGnBu_09.scale(min(df_pickup['Count']), max(df_pickup['Count']))

In [17]:
#apply logarithmic transformation, since count values are highly distributed between 1.0 and ca. 190.000.
#Logarithmic transformation helps with visualization of Count differences
#Furhter scaling the number puts them in a range between 0 and 1, which furhter helps with visualization
df_pickup['Count'] = np.log1p(df_pickup['Count'])
#df_pickup['Count'] = scaler.fit_transform(df_pickup[['Count']])

In [18]:
min_value_pick = df_pickup['Count'].min()
max_value_pick = df_pickup['Count'].max()

In [19]:
m_pick = folium.Map(location=[41.8781, -87.6298], zoom_start=12)

In [20]:
cm_pick = linear.YlGnBu_09.scale(min_value_pick, max_value_pick)

In [21]:
list_features = []
#Convert polygons to geojson data, so we can visualize it
    
for i in range(len(df_pickup)):
    
    count = df_pickup.iloc[i]['Count']
    feature = Feature(
        geometry= df_pickup.iloc[i]['Geometry'],
        id=df_pickup.iloc[i]['pickup_hex'],
        properties={"resolution": resolution, 'count': count}
    )
    list_features.append(feature)

feat_collection = FeatureCollection(list_features)
geojson_result1 = json.dumps(feat_collection)

In [22]:
GeoJson(
        geojson_result1,
        style_function = lambda feature: {
            'fillColor': cm_pick(float(feature['properties']['count'])),
            'color': cm_pick(float(feature['properties']['count'])),
            'weight': 2,
            'fillOpacity': 0.5
        },
        name = "Example"
    ).add_to(m_pick)
legend_pick.add_to(m_pick)

## Areas by Pickup

In [23]:
m_pick

In [ ]:
#doing the same for dropoffs

In [33]:
#input dataframe has to contain
def visualize_data(df, column, resolution, color):
    
        count = df.groupby(by= column).agg({'Count': vaex.agg.count(column)}) 
        count.sort(by='Count', ascending= False)
        print("Checkpoint1")
        count["Geometry"] = count[column].apply(
        h3_to_poly)
        
        count = count.to_pandas_df()
        count['Count'] =count['Count'].astype("float")
        
        legend = color.scale(min(count['Count']), max(count['Count']))
        
        count['Count_scaled'] = np.log1p(df['Count'])
        
        cm = linear.YlOrRd_09.scale(min(count['Count_scaled']), max(count['Count_scaled']))
        
        m = folium.Map(location=[41.8781, -87.6298], zoom_start=11)

        list_features = []

        for i in range(1, len(count)):
            count_var = count.iloc[i]['Count']
            feature = Feature(geometry= count.iloc[i]['Geometry'],
                              id = count.iloc[i][column],
                              properties = {"resolution": resolution, "count": count_var })
            list_features.append(feature)


            feat_collection = FeatureCollection(list_features)
            geojson_result = json.dumps(feat_collection)


        GeoJson(
            geojson_result,
            style_function = lambda feature: {
          
                'color': cm_drop(float(feature['properties']['count'])),
                'weight': 2,
                'fillOpacity': 0.5,
                'fillColor': cm_drop(float(feature['properties']['count'])),
            },
            name = "Example"
        ).add_to(m)
        legend.add_to(m)
        
        return m

In [34]:
map1 = visualize_data(df, "pickup_hex", 7, "YlOrRd_09")

KeyboardInterrupt: 

In [28]:
map1

(#           trip_id                                   taxi_id                                              trip_start_timestamp        trip_end_timestamp          trip_seconds    trip_miles    pickup_census_tract    dropoff_census_tract    pickup_community_area    dropoff_community_area    fare    tips    tolls    extras    trip_total    payment_type    company                            pickup_centroid_latitude    pickup_centroid_longitude    dropoff_centroid_latitude    dropoff_centroid_longitude    trip_start_hour    trip_end_hour    trip_start_4h_period    trip_end_4h_period    pickup_hex
 0           1b545d0be27b195076e5d2a7d4fc0ebef6ef20ac  '56a1119c6ca57e39525cf06829f9ecff553cf4b5ac24821...  2017-01-01 00:00:00.000000  2017-01-01 00:00:00.000000  240.0           1.3           17031839100.0          17031833000.0           32.0                     28.0                      6.25    0.0     0.0      0.0       6.25          Cash            Taxi Affiliation Services          41.8809